In [1]:
import copy

class FragileDict:
    def __init__(self, _data={}):
        self._data = copy.deepcopy(_data)
        self._lock = 0
    
    def __setitem__(self, X, y):
        if self._lock:
            self.copy[X] = y
        else:
            raise RuntimeError("Protected state")

    def __enter__(self):
        self._lock = 1
        self.copy = copy.deepcopy(self._data)
        self.backup = copy.deepcopy(self._data)
        return self
            
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is not None:
            print("Exception has been suppressed.")
            self._data = self.backup
            delattr(self, 'copy')
            delattr(self, 'backup')
            self._lock = 0
            return True
        self._data = copy.deepcopy(self.copy)
        delattr(self, 'copy')
        delattr(self, 'backup')
        self._lock = 0
        return True
    
    def __getitem__(self, X):        
        if self._lock:
            return self.copy[X]
        else:
            return copy.deepcopy(self._data[X])
        
    def __contains__(self, X):
        if self._lock:
            return X in self.copy.keys()
        else:
            return X in self._data.keys()